In [ ]:
import cv2 
import mediapipe as mp
import pyautogui

In [ ]:
# Initialize hand detection 
mp_hands = mp.solutions.hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

pyautogui.FAILSAFE = False

# define hand landmark to track
wrist_landmark_id = mp.solutions.hands.HandLandmark.WRIST
index_landmark_id = mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP

# get screen size
screen_width, screen_height = pyautogui.size()

# sensitivity factor for cursor movement (adjust for desired speed)
sensivity = 5

# Initialize previous landmark positions
prev_fingertip_x = 0
prev_fingertip_y = 0




    
    
    
    


In [ ]:

        
def detect_hands(frame):
  """
  Detects hands in a frame using MediaPipe.

  Args:
      frame: The frame to be processed.

  Returns:
      A list of hand landmarks if a hand is detected, otherwise None.
  """
  rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for MediaPipe
  results = mp_hands.process(rgb_frame)
  if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]  # Assuming only one hand
    return hand_landmarks
  else:
    return None

while True:
  # Take a picture with the camera
  success, frame = cap.read()

  # 2. Spot the Tips (continued) - Find the fingertip in this picture
  hand_landmarks = detect_hands(frame)  # This function is defined later

  # If we found a fingertip, let's move the cursor!
  if hand_landmarks:
    # Remember where the fingertip is now
    fingertip_landmark = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
    fingertip_x = int(fingertip_landmark.x * screen_width)
    fingertip_y = int(fingertip_landmark.y * screen_height)

    # 3. Track the Movement (continued) - Remember where it was before
    movement_x = fingertip_x - prev_fingertip_x
    movement_y = fingertip_y - prev_fingertip_y

    # Make the movement smoother (like a steady remote control)
    smoothing_factor = 0.5
    smoothed_movement_x = smoothing_factor * movement_x + (1 - smoothing_factor) * prev_fingertip_x
    smoothed_movement_y = smoothing_factor * movement_y + (1 - smoothing_factor) * prev_fingertip_y

    # 4. Move the Cursor (continued) - Move the mouse pointer based on finger movement
    pyautogui.moveRel(smoothed_movement_x, smoothed_movement_y, duration=0.01)

    # Remember where the fingertip is for next time
    prev_fingertip_x = smoothed_movement_x
    prev_fingertip_y = smoothed_movement_y

  # Show the picture with the fingertip (optional)
  # cv2.imshow('Hand', frame)

  # Close the program if you press 'Esc'
  if cv2.waitKey(5) & 0xFF == 27:
    break

# Clean up the camera
cap.release()
cv2.destroyAllWindows()



In [ ]:
main()